<a href="https://colab.research.google.com/github/imabari/rakuten_tools/blob/main/rakuten_ta_ehime_aria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 設定

In [1]:
# 1TAの距離
N = 150

In [2]:
# 緯度,経度,TA

data = """\
33.464151,132.448912,9
33.470405,132.462822,2
33.465314,132.454744,6
"""

# プログラム

In [3]:
!pip install folium

In [4]:
import io
import pandas as pd

In [5]:
import folium

In [6]:
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vRE1NoYtNw1FmjRQ8wcdPkcE0Ryeoc2mfFkCQPHjzwL5CpwNKkLXnBl_F7c0LZjrtbLtRLH55ZVi6gQ/pub?gid=0&single=true&output=csv"

df0 = (
    pd.read_csv(url, index_col=0, usecols=[0, 1, 2, 6, 7, 10, 11, 12, 13])
    .dropna(how="all")
    .fillna("")
)

df0["color"] = df0["状況"].replace({"open": "green", "close": "red", "ready": "orange"})
df0["場所"] = df0["場所"].str.strip()

In [7]:
df0

,場所,場所補足,状況,sector,eNB-LCID,PCI,緯度,経度,color
番号,,,,,,,,,
1,今治市立花町2丁目,,open,3,"737335-1,2,3","39,347,139",34.052319,132.999271,green
2,今治市中寺①,今治中寺郵便局付近,open,3,"737331-1,2,3","97,98,96",34.037485,132.991341,green
3,今治市桜井4丁目,志島ヶ原・綱敷天満神社付近,open,3,"737342-7,8,9","239,222,100",34.017402,133.041459,green
4,今治市登畑,レデイ薬局今治国分店付近,open,3,"737352-10,11,12","136,137,135",34.019522,133.016295,green
5,今治市阿方①,ファミリーマート今治阿方店付近,open,3,"737354-4,5,6","225,227,226",34.061198,132.970841,green
...,...,...,...,...,...,...,...,...,...
348,南宇和郡愛南町御荘平城①,ローソン愛南町御荘平城店,close,3,,,32.965683,132.550687,red
349,南宇和郡愛南町御荘平城②,長州園地,close,3,,,32.968995,132.544405,red
350,南宇和郡愛南町城辺甲沖,,close,,,,32.960264,132.575333,red


In [8]:
df1 = pd.read_csv(io.StringIO(data), header=None, names=["lat", "lng", "ta"])

In [9]:
map = folium.Map(
    tiles=None,
    location=[df1["lat"].mean(), df1["lng"].mean()],
    zoom_start=12,
)

In [10]:
folium.raster_layers.TileLayer(
    tiles="https://cyberjapandata.gsi.go.jp/xyz/pale/{z}/{x}/{y}.png",
    name='国土地理院',
    attr='&copy; <a href="https://maps.gsi.go.jp/development/ichiran.html">国土地理院</a>'
).add_to(map)

In [11]:
# Googleマップ標準
folium.raster_layers.TileLayer(
    "https://{s}.google.com/vt/lyrs=m&x={x}&y={y}&z={z}",
    subdomains=["mt0", "mt1", "mt2", "mt3"],
    name="Google Map",
    attr="<a href='https://developers.google.com/maps/documentation' target='_blank'>Google Map</a>",
).add_to(map)

In [12]:
folium.raster_layers.TileLayer(
    name="楽天モバイル",
    tiles='https://gateway-api.global.rakuten.com/dsd/geoserver/4g4m/mno_coverage_map/gwc/service/gmaps?LAYERS=mno_coverage_map:all_map&FORMAT=image/png&TRANSPARENT=TRUE&x={x}&y={y}&zoom={z}',
    fmt='image/png',
    attr="楽天モバイルエリア",
    tms=False,
    overlay=True,
    control=True,
    opacity=1,
).add_to(map)

In [13]:
folium.raster_layers.TileLayer(
    name="楽天モバイル5G",
    tiles="https://gateway-api.global.rakuten.com/dsd/geoserver/5g/mno_coverage_map/gwc/service/gmaps?LAYERS=mno_coverage_map:all_map&FORMAT=image/png&TRANSPARENT=TRUE&x={x}&y={y}&zoom={z}",
    fmt="image/png",
    attr="楽天モバイルエリア",
    tms=False,
    overlay=True,
    control=True,
    opacity=1,
    show=False,
).add_to(map)

# map

In [14]:
fg0 = folium.FeatureGroup(name="基地局").add_to(map)
fg1 = folium.FeatureGroup(name="観測地点").add_to(map)
fg2 = folium.FeatureGroup(name="TA").add_to(map)

In [15]:
for i, r in df0.iterrows():

    fg0.add_child(
        folium.Marker(
            location=[r["緯度"], r["経度"]],
            popup=folium.Popup(f'<p><a href="https://www.google.com/maps?layer=c&cbll={r["緯度"]},{r["経度"]}">{r["場所"]}</a></p>', max_width=300),
            tooltip=f'{r["場所"]}',
            icon=folium.Icon(color=r["color"], icon="signal"),
        )
    )

In [16]:
for i, r in df1.iterrows():

    fg1.add_child(
        folium.Marker(
            location=[r.lat, r.lng],
            icon=folium.Icon(color="blue"),
        )

    )
    fg2.add_child(
            folium.Circle(
            location=[r.lat, r.lng],
            radius=r.ta * N,
            color="yellow",
        )
    )

In [17]:
folium.LayerControl().add_to(map)

In [18]:
from folium import plugins

In [19]:
# DRAW
folium.plugins.Draw(draw_options = {'polygon': False,'rectangle': False,'circlemarker': False}).add_to(map)

In [20]:
map

In [21]:
map.save("map.html")